# Getting started with the Keras functional API

reference: https://keras.io/getting-started/functional-api-guide/

## First Example: Fully Connected Network

In [1]:
from keras.layers import Input, Dense
from keras.models import Model

Using Theano backend.


In [9]:
# this returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Multi-input and multi-output models

![multi-input-multi-output-graph.png](./figures/multi-input-multi-output-graph.png)

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense, merge
from keras.models import Model

In [14]:
main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)
aux_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

aux_input = Input(shape=(5,), name='aux_input')
x = merge([lstm_out, aux_input], mode='concat')
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

Since our inputs and outputs are named (we passed them a "name" argument), We could also have compiled the model via

    total_loss = 1.0*loss_of_main + 0.2*loss_of_aux

In [19]:
model = Model(input=[main_input, aux_input], output=[main_output, aux_output])
model.compile(optimizer='adam', 
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights = [1.0, 0.2]
             )
print model.summary()

# than you can fit model
# model.fit([headline_data, additional_data], [labels, labels],
#          nb_epoch=50, batch_size=32)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 100)           0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 100, 512)      5120000     main_input[0][0]                 
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 32)            69760       embedding_4[0][0]                
____________________________________________________________________________________________________
aux_input (InputLayer)           (None, 5)             0                                            
___________________________________________________________________________________________

## Shared layers

In [23]:
from keras.layers import Input, LSTM, Dense, merge
from keras.models import Model

input_a = Input(shape=(140, 256))
input_b = Input(shape=(140, 256))

# input_a and input_b shared the same layer
shared_lstm = LSTM(64)
encoded_a = shared_lstm(input_a)
encoded_b = shared_lstm(input_b)

## The concept of layer "node"

Whenever you are calling a layer on some input, you are creating a new tensor (the output of the layer), and you are adding a "node" to the layer, linking the input tensor to the output tensor. When you are calling the same layer multiple times, that layer owns multiple nodes indexed as 0, 1, 2...

In [33]:
print shared_lstm.get_output_at(0)
print shared_lstm.get_input_at(0)
print shared_lstm.get_output_at(1)
print shared_lstm.get_input_at(1)

print shared_lstm.get_input_shape_at(0)
print shared_lstm.get_input_shape_at(1)
print shared_lstm.get_output_shape_at(0)

Subtensor{int64}.0
input_11
Subtensor{int64}.0
input_12
(None, 140, 256)
(None, 140, 256)
(None, 64)
